This notebook is for calculating distances, elevations and timings between Dovetrek checkpoints. It then produces a routecard for a set route.

###Contents:
* [Imports](#Imports)
* [Start SparkSession](#Start-SparkSession)
* [Set Parameters](#set-params)
* [Get Secrets](#get-secrets)
* [Fetch Checkpoint Information](#fetch-cp-info)
* [Fetch Distances from Filestore](#)
* [Calculate leg timings for each Checkpoint Combination](#cp-combo-times)
* [Pick a route](#pick-route)
* [Calculate route timings](#route-timings)
* [Calculate checkpoint arrival and departure times for route](#route-times)

### Imports

In [0]:
from pyspark.sql.functions import col, expr, first
from pyspark.sql import functions as F, DataFrame, Window, SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, FloatType, ArrayType, TimestampType
from datetime import datetime, time, timedelta
import requests
import time
import ipywidgets
import base64
import getpass
import builtins
import json
from io import BytesIO, StringIO
import itertools
from functools import reduce
import numpy as np

### Start SparkSession

In [0]:
try:
    SparkSession.active()
except:
    spark = SparkSession.builder \
        .appName("BinderPySpark") \
        .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
        .getOrCreate()
else:
    SparkSession.active()
finally:
    # Test if Spark is working
    df = spark.createDataFrame([(1, "Alice"), (2, "Bob")], ["id", "name"])
    df.show()
    df.unpersist()

+---+-----+
| id| name|
+---+-----+
|  1|Alice|
|  2|  Bob|
+---+-----+



### Set Parameters

In [0]:
Year_List = ["2025", "2024", "2019", "2018", "2017"]
API_Service_List = ["Bing Maps","Google Maps", "OpenRouteService","Azure Maps & OpenTopoData"]

try:
  dbutils.widgets.dropdown("Competition_Year", "2025", Year_List)
  Competition_Year = dbutils.widgets.get("Competition_Year")

  dbutils.widgets.dropdown("API_Service", "Google Maps", API_Service_List)
  API_Service = dbutils.widgets.get("API_Service")

  dbutils.widgets.text("Dwell", "7")
  StopTimeAtCheckPoints = dbutils.widgets.get("Dwell")

  dbutils.widgets.text("Speed", "5.30")
  Speed = dbutils.widgets.get("Speed")

  dbutils.widgets.text("StartTime", "10:00:00")
  StartTime =  datetime.strptime(dbutils.widgets.get("StartTime"), '%H:%M:%S')

  dbutils.widgets.text("Naismith", "10")
  AddMinutesPer100mHeight = dbutils.widgets.get("Naismith")

except:
  Competition_Year_Picker = ipywidgets.Dropdown(options=Year_List, value = '2024', description = 'Pick a competition year:')
  API_Service_Picker = ipywidgets.Dropdown(options=API_Service_List, value = "Google Maps", description = 'Pick an API Service for distance & elevations:')
  StopTimePicker = ipywidgets.IntSlider(value = 7
                                            , min = 0
                                            , max= 20
                                            , description = 'Pick a time in minutes to stop at checkpoints'
                                            , readout = True
                                          , step = 1)
  NaismithPicker = ipywidgets.IntSlider(value = 10
                                            , min = 0
                                            , max= 20
                                            , description = 'Add minutes per 100m elevation gain'
                                            , readout = True
                                          , step = 1)
  SpeedPicker = ipywidgets.FloatSlider(
                                          value=5.3,
                                          min=0,
                                          max=10.0,
                                          step=0.1,
                                          description='Hiking speed:',
                                          disabled=False,
                                          continuous_update=False,
                                          orientation='horizontal',
                                          readout=True,
                                          readout_format='.1f',
                                      )
  StartTimePicker = ipywidgets.Text(
                                              description='Type a StartTime in the format HH:MM:SS',
                                              value = '10:00:00',
                                              disabled=False
                                          )
  display(Competition_Year_Picker)
  display(API_Service_Picker)
  display(StopTimePicker)
  display(NaismithPicker)
  display(StartTimePicker)
  display(SpeedPicker)

  input("Fill and ctrl + enter when all parameters are set to the desired state")

  Competition_Year = Competition_Year_Picker.value
  API_Service = API_Service_Picker.value
  StopTimeAtCheckPoints = StopTimePicker.value
  AddMinutesPer100mHeight = NaismithPicker.value
  Speed = SpeedPicker.value
  StartTime = datetime.strptime(StartTimePicker.value, '%H:%M:%S')

finally:

  FinishTime = StartTime + timedelta(hours=7)
  print(f"Competition Year: {Competition_Year} \nAPI Service: {API_Service} \nStopTimeAtCheckPoints: {StopTimeAtCheckPoints} minutes \nAddMinutesPer100mHeight: {AddMinutesPer100mHeight} minutes \nSpeed: {Speed} km/h \nStartTime: {StartTime}")


Competition Year: 2025 
API Service: Azure Maps & OpenTopoData 
StopTimeAtCheckPoints: 7 minutes 
AddMinutesPer100mHeight: 10 minutes 
Speed: 5.30 km/h 
StartTime: 1900-01-01 10:00:00


### Get Secrets

In [0]:
GitHubPAT = getpass.getpass("Enter your GitHub Secrets Repo PAT: ")  # Prompt for PAT if not set

# 🔹 GitHub API URL to get the file
url = f"https://api.github.com/repos/liamj-f/Secrets/contents/Dovetrek_Secrets.json"

# 🔹 GitHub API headers
headers = {"Authorization": f"token {GitHubPAT}"}

# 🔹 Fetch the secrets file
response = requests.get(url, headers=headers)

if response.status_code == 200:
    # Decode Base64 content
    content = response.json()["content"]
    decoded_content = base64.b64decode(content).decode("utf-8")

    # Load JSON into a dictionary
    secrets = json.loads(decoded_content)

    # 🔹 Assign values to variables
    BingMapsAPIKey = secrets.get("BingMApsAPIKey", "")
    AzureMapsAPIKey = secrets.get("AzureMapsAPIKey", "")
    OrdnanceSurveyAPIKey = secrets.get("OrdnanceSurveyAPIKey", "")
    OpenRouteServiceAPIKey = secrets.get("OpenRouteServiceAPIKey", "")
    GoogleMapsAPIKey = secrets.get("GoogleMapsAPIKey", "")
    DovetrekRepoPAT = secrets.get("DovetrekRepoPAT", "")

    print("✅ Secrets loaded successfully!")

else:
    print(f"❌ Error: {response.status_code} - {response.text}")


Enter your GitHub Secrets Repo PAT:  [REDACTED]

✅ Secrets loaded successfully!


### Fetch Checkpoint information

In [0]:
# GitHub API URL for file content
url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/CheckpointData/Openings_{Competition_Year}.csv"

# Fetch file content
response = requests.get(url)
content = response.json()["content"]
decoded_content = base64.b64decode(content).decode("utf-8")
csv_lines = decoded_content.split("\n")
# Create an RDD from the list
rdd = spark.sparkContext.parallelize(csv_lines)
# Convert RDD to DataFrame
openings_df = spark.read.csv(rdd, header=True, inferSchema=True)

### List Filestore contents

In [0]:
url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/DataFrames"

params = {"ref": "FileStore", 
          "Authorization": DovetrekRepoPAT}

response = requests.get(url, params=params, headers=headers)

response.json()



[{'name': 'Distances_DF_2025_Azure Maps & OpenTopoData_2025-02-23.csv',
  'path': 'DataFrames/Distances_DF_2025_Azure Maps & OpenTopoData_2025-02-23.csv',
  'sha': '7d1d3a863ac353970da56c8576bc6fcb64642c78',
  'size': 8170,
  'url': 'https://api.github.com/repos/liamj-f/Dovetrek/contents/DataFrames/Distances_DF_2025_Azure%20Maps%20&%20OpenTopoData_2025-02-23.csv?ref=FileStore',
  'html_url': 'https://github.com/liamj-f/Dovetrek/blob/FileStore/DataFrames/Distances_DF_2025_Azure%20Maps%20&%20OpenTopoData_2025-02-23.csv',
  'git_url': 'https://api.github.com/repos/liamj-f/Dovetrek/git/blobs/7d1d3a863ac353970da56c8576bc6fcb64642c78',
  'download_url': 'https://raw.githubusercontent.com/liamj-f/Dovetrek/FileStore/DataFrames/Distances_DF_2025_Azure%20Maps%20%26%20OpenTopoData_2025-02-23.csv',
  'type': 'file',
  '_links': {'self': 'https://api.github.com/repos/liamj-f/Dovetrek/contents/DataFrames/Distances_DF_2025_Azure%20Maps%20&%20OpenTopoData_2025-02-23.csv?ref=FileStore',
   'git': 'http

### Get the Latest File

In [0]:
import fnmatch
import urllib
# Define the pattern with wildcard
pattern = "Distances_DF_2025_Azure Maps & OpenTopoData_*.csv"

# Filter files matching the pattern
matched_files = [file for file in response.json() if fnmatch.fnmatch(file["name"], pattern)]

# Sort by date in filename (YYYY-MM-DD at the end)
matched_files.sort(key=lambda x: x["name"].split("_")[-1], reverse=True)

# Get the latest matching file
latest_file = matched_files[0] if matched_files else None

if latest_file:
    print("Latest matching file:", latest_file["name"])
    print("Download URL:", latest_file["download_url"])
else:
    print("No matching files found.")

latest_file = latest_file['name']
latest_file = urllib.parse.quote(latest_file)


Latest matching file: Distances_DF_2025_Azure Maps & OpenTopoData_2025-02-23.csv
Download URL: https://raw.githubusercontent.com/liamj-f/Dovetrek/FileStore/DataFrames/Distances_DF_2025_Azure%20Maps%20%26%20OpenTopoData_2025-02-23.csv


### Fetch Distances_DF csv to GitHub filestore

In [0]:

url = f"https://api.github.com/repos/liamj-f/Dovetrek/contents/DataFrames/{latest_file}"

params = {"ref": "FileStore"}
# Get the latest SHA (if the file exists)
response = requests.get(url, params=params)
content = response.json()["content"]
decoded_content = base64.b64decode(content).decode("utf-8")
csv_lines = decoded_content.split("\n")
# Create an RDD from the list
rdd = spark.sparkContext.parallelize(csv_lines)
# Convert RDD to DataFrame
Distances_DF = spark.read.csv(rdd, header=True, inferSchema=True)

display(Distances_DF)

StartCP,FinishCP,Distance,Height_Gain
CP1,CP2,1.997,47.820038
CP1,CP3,3.288,81.472595
CP1,CP4,10.511,260.00488
CP1,CP5,5.531,82.841
CP1,CP6,4.331,47.787582
CP1,CP7,0.458,1.0333252
CP1,E1,0.956,32.138275
CP1,E10U,2.758,107.77533
CP1,E2R,3.405,135.88821
CP1,E3,3.515,5.1418915


### Calculate leg timings for each Checkpoint Combination

In [0]:
Distances_DF = Distances_DF.withColumn("TimeInMinutes",(col("Distance")/ Speed)*60 + (col("Height_Gain") / AddMinutesPer100mHeight ) )
display(Distances_DF)

StartCP,FinishCP,Distance,Height_Gain,TimeInMinutes
CP1,CP2,1.997,47.820038,27.389550969811324
CP1,CP3,3.288,81.472595,45.36990100943396
CP1,CP4,10.511,260.00488,144.99294083018867
CP1,CP5,5.531,82.841,70.89919433962264
CP1,CP6,4.331,47.787582,53.80894687924529
CP1,CP7,0.458,1.0333252,5.288238180377359
CP1,E1,0.956,32.138275,14.036469009433961
CP1,E10U,2.758,107.77533,42.00017450943397
CP1,E2R,3.405,135.88821,52.13599081132075
CP1,E3,3.515,5.1418915,40.30664198018868


### Pick a Route

In [0]:
# Define the route and start time
route = ["Start","CP3", "E7U", "CP4", "E8", "E6", "CP5", "E5", "E4R", "CP6", "E9U","CP2","E2R","E10U", "E1", "CP1", "CP7", "E3", "Finish"]

### Calculate route timings

In [0]:
# Create a DataFrame for the route sequence
route_df = spark.createDataFrame([(route[i], route[i+1], i) for i in range(len(route)-1)],
                                 ["StartCP", "FinishCP", "RouteOrder"])

# Join with Distances_Df to get TimeInMinutes for each leg
timing_results_df = (route_df
      .join(Distances_DF, ["StartCP", "FinishCP"], "left")
      .orderBy("RouteOrder"))

# Use a window function to calculate cumulative time
window_spec = Window.orderBy("RouteOrder").rowsBetween(Window.unboundedPreceding, 0)
timing_results_df = timing_results_df.withColumn("CumulativeTime", F.sum("TimeInMinutes").over(window_spec))

# Show results
display(timing_results_df)

StartCP,FinishCP,RouteOrder,Distance,Height_Gain,TimeInMinutes,CumulativeTime
Start,CP3,0,1.677,59.83551,24.968456660377363,24.968456660377363
CP3,E7U,1,6.751,133.96097,89.82251209433961,114.79096875471697
E7U,CP4,2,0.473,44.57132,9.811848981132076,124.60281773584904
CP4,E8,3,2.035,5.496788,23.587414649056605,148.19023238490564
E8,E6,4,2.164,58.170288,30.315142007547173,178.5053743924528
E6,CP5,5,1.945,102.13289,32.232156924528304,210.73753131698112
CP5,E5,6,4.746,50.215477,58.749849586792465,269.48738090377356
E5,E4R,7,1.42,83.2854,24.40401169811321,293.89139260188676
E4R,CP6,8,0.985,22.702316,13.421174996226416,307.31256759811316
CP6,E9U,9,1.137,73.60889,20.232587113207547,327.5451547113207


### Calculate checkpoint arrival and departure times for route

In [0]:
# Define window specification to order rows by RouteOrder
window_spec = Window.orderBy("RouteOrder")

# Calculate the CumulativeTimeWithDwell (CumulativeTime + StopTimeAtCheckpoints)
timing_results_df = timing_results_df.withColumn(
    "CumulativeTimeWithDwell",
    F.col("CumulativeTime") + (F.col("RouteOrder") * F.lit(StopTimeAtCheckPoints))
)

# Calculate the LeavingTime (StartTime + CumulativeTimeWithDwell)
timing_results_df = timing_results_df.withColumn(
    "LeavingTime",
    F.to_timestamp(F.lit(StartTime)) + F.col("CumulativeTimeWithDwell").cast("int").cast("interval minute")
)

# Calculate the ArrivalTime (LeavingTime - StopTimeAtCheckpoints)
timing_results_df = timing_results_df.withColumn(
    "ArrivalTime",
    F.col("LeavingTime") - F.expr(f"INTERVAL {StopTimeAtCheckPoints} MINUTE")
)

# Show the results
display(timing_results_df)


StartCP,FinishCP,RouteOrder,Distance,Height_Gain,TimeInMinutes,CumulativeTime,CumulativeTimeWithDwell,LeavingTime,ArrivalTime
Start,CP3,0,1.677,59.83551,24.968456660377363,24.968456660377363,24.968456660377363,1900-01-01T10:24:00Z,1900-01-01T10:17:00Z
CP3,E7U,1,6.751,133.96097,89.82251209433961,114.79096875471697,121.79096875471697,1900-01-01T12:01:00Z,1900-01-01T11:54:00Z
E7U,CP4,2,0.473,44.57132,9.811848981132076,124.60281773584904,138.60281773584904,1900-01-01T12:18:00Z,1900-01-01T12:11:00Z
CP4,E8,3,2.035,5.496788,23.587414649056605,148.19023238490564,169.19023238490564,1900-01-01T12:49:00Z,1900-01-01T12:42:00Z
E8,E6,4,2.164,58.170288,30.315142007547173,178.5053743924528,206.5053743924528,1900-01-01T13:26:00Z,1900-01-01T13:19:00Z
E6,CP5,5,1.945,102.13289,32.232156924528304,210.73753131698112,245.73753131698112,1900-01-01T14:05:00Z,1900-01-01T13:58:00Z
CP5,E5,6,4.746,50.215477,58.749849586792465,269.48738090377356,311.48738090377356,1900-01-01T15:11:00Z,1900-01-01T15:04:00Z
E5,E4R,7,1.42,83.2854,24.40401169811321,293.89139260188676,342.89139260188676,1900-01-01T15:42:00Z,1900-01-01T15:35:00Z
E4R,CP6,8,0.985,22.702316,13.421174996226416,307.31256759811316,363.31256759811316,1900-01-01T16:03:00Z,1900-01-01T15:56:00Z
CP6,E9U,9,1.137,73.60889,20.232587113207547,327.5451547113207,390.5451547113207,1900-01-01T16:30:00Z,1900-01-01T16:23:00Z
